In [ ]:
import tensorflow as tf
import random
import numpy as np
from keras import datasets, models, layers, optimizers
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
import random

seed_value = 42
np.random.seed(seed_value)
random.seed(seed_value)
tf.random.set_seed(seed_value)

(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1))
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1))

train_images = x_train.astype('float32') / 255
test_images = x_test.astype('float32') / 255
train_labels = to_categorical(y_train)
test_labels = to_categorical(y_test)
val_images = train_images[:10000]
partial_images = train_images[10000:]
val_labels = train_labels[:10000]
partial_labels = train_labels[10000:]

number_of_epochs = 10

def create_and_train_model(params):
    num_layers = params[0]
    num_filters_params = params[1:num_layers + 1]
    kernel_sizes_params = params[num_layers + 1:2 * num_layers + 1]

    dropout_rate_cnn = 0.1
    dropout_rate_mlp = 0.5
    optimizer = optimizers.Adam()
    activation_function = 'relu'

    model = models.Sequential()
    input_shape = (28, 28, 1)
    num_classes = 10
    num_epochs = number_of_epochs

    model.add(layers.Conv2D(num_filters_params[0], (kernel_sizes_params[0], kernel_sizes_params[0]), padding='same', input_shape=input_shape, activation=activation_function))
    model.add(layers.BatchNormalization())

    for i in range(1, num_layers):
        kernel_size = kernel_sizes_params[i]
        if kernel_size > model.output_shape[1]:
            kernel_size = model.output_shape[1]

        model.add(layers.Conv2D(num_filters_params[i], (kernel_size, kernel_size), padding='same', activation=activation_function))
        model.add(layers.BatchNormalization())

        if i % 2 == 1 and model.output_shape[1] > 2 and model.output_shape[2] > 2:
            model.add(layers.MaxPooling2D(2, 2))

    if model.output_shape[1] <= 0 or model.output_shape[2] <= 0:
        raise ValueError(f"Output dimensions are invalid: {model.output_shape}")

    model.add(layers.Flatten())
    model.add(layers.Dropout(dropout_rate_cnn))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(dropout_rate_mlp))
    model.add(layers.Dense(num_classes, activation="softmax"))

    model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

    es = EarlyStopping(monitor="val_accuracy", patience=3)
    history = model.fit(partial_images, partial_labels, validation_data=(val_images, val_labels), epochs=num_epochs, callbacks=[es], verbose=0)

    return params, history.history['val_accuracy'][-1]


def generate_binary_list(length):
    return [random.randint(0, 1) for _ in range(length)]

def one_point_crossover(parent1, parent2):
    pivot = random.randint(1, min(len(parent1), len(parent2)) - 1)
    offspring1 = parent1[:pivot] + parent2[pivot:]
    offspring2 = parent2[:pivot] + parent1[pivot:]
    offspring1[0] = len(offspring1[1:]) // 2
    offspring2[0] = len(offspring2[1:]) // 2
    return offspring1, offspring2

def crossover(parent1, parent2, current_generation, total_generations):
    prob_binary_crossover = 1 - current_generation / total_generations
    prob_one_point_crossover = 1 - prob_binary_crossover

    if random.random() < prob_binary_crossover:
        n = len(parent1)
        m = len(parent2)
        binary_list = generate_binary_list(max(n, m) - 1)

        offspring1 = [parent1[0]]
        offspring2 = [parent2[0]]

        for i in range(1, min(n, m)):
            if binary_list[i-1] == 0:
                offspring1.append(parent1[i])
                offspring2.append(parent2[i])
            else:
                offspring1.append(parent2[i])
                offspring2.append(parent1[i])

        if n > m:
            offspring1.extend(parent1[m:])
        elif m > n:
            offspring2.extend(parent2[n:])

    else:
        offspring1, offspring2 = one_point_crossover(parent1, parent2)

    return offspring1, offspring2

def mutate_individual(individual, mutation_rate=0.3):
    if random.random() < mutation_rate:
        mutation_type = random.choice(['allele', 'layers'])
        if mutation_type == 'allele':
            idx = random.randint(1, len(individual) - 1)
            if idx % 2 == 0:
                individual[idx] = random.randint(2, 9)
            else:
                individual[idx] = random.randint(4, 64)
        else:
            new_n = random.randint(2, 6)
            old_n = individual[0]
            individual[0] = new_n
            if new_n < old_n:
                individual = individual[:-(2 * (old_n - new_n))]
            elif new_n > old_n:
                additional_genes = []
                for _ in range(2 * (new_n - old_n)):
                    if len(additional_genes) % 2 == 0:
                        additional_genes.append(random.randint(4, 128))
                    else:
                        additional_genes.append(random.randint(2, 9))
                individual.extend(additional_genes)
    return individual

def mutate_all_pop(pop, score):
    new_pop = []
    new_score = []

    for i in range(len(pop)):
        original_individual = pop[i].copy()
        mutated_individual = mutate_individual(pop[i])

        if original_individual != mutated_individual:
            new_pop.append(mutated_individual)
            _, new_fitness = create_and_train_model(mutated_individual)
            new_score.append(new_fitness)

    pop.extend(new_pop)
    score.extend(new_score)

def tournament_selection(population, scores, k=2):
    selected = random.sample(list(zip(population, scores)), k)
    selected.sort(key=lambda x: x[1], reverse=True)
    return selected[0][0]

def create_children(population, scores, number_of_children, current_generation, total_generations):
    number_of_children = number_of_children // 2

    for _ in range(number_of_children):
        parent1 = tournament_selection(population, scores)
        parent2 = tournament_selection(population, scores)

        child1, child2 = crossover(parent1, parent2, current_generation, total_generations)

        child1 = mutate_individual(child1)
        child2 = mutate_individual(child2)

        try:
            a, b = create_and_train_model(child1)
            population.append(child1)
            scores.append(b)
        except Exception as e:
            print(f"Error training child1: {e}")
            continue

        try:
            c, d = create_and_train_model(child2)
            population.append(child2)
            scores.append(d)
        except Exception as e:
            print(f"Error training child2: {e}")
            continue

def rank_based_selection(population, scores, num_to_keep):
    ranked_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)

    total_rank = sum(range(1, len(ranked_indices) + 1))
    probabilities = [(len(ranked_indices) - rank) / total_rank for rank in range(len(ranked_indices))]

    selected_indices = set()
    while len(selected_indices) < num_to_keep:
        selected_index = random.choices(ranked_indices, weights=probabilities, k=1)[0]
        selected_indices.add(selected_index)

    selected_indices = list(selected_indices)
    population[:] = [population[i] for i in selected_indices]
    scores[:] = [scores[i] for i in selected_indices]

def fitness_based_selection(population, scores, num_to_keep):
    total_fitness = sum(scores)
    probabilities = [score / total_fitness for score in scores]

    selected_indices = set()
    while len(selected_indices) < num_to_keep:
        selected_index = random.choices(range(len(population)), weights=probabilities, k=1)[0]
        selected_indices.add(selected_index)

    selected_indices = list(selected_indices)
    population[:] = [population[i] for i in selected_indices]
    scores[:] = [scores[i] for i in selected_indices]

def survivor_selection(population, scores, num_survivors, generation, total_generations):
    probability_fitness_based = 1 - generation / total_generations
    probability_rank_based = 1 - probability_fitness_based

    if random.random() < probability_fitness_based:
        fitness_based_selection(population, scores, num_survivors)
    else:
        rank_based_selection(population, scores, num_survivors)

def generate_random_hyperparameters():
    first_gene = random.randint(2, 6)
    chromosome_length = 1 + 2 * first_gene
    chromosome = [first_gene]
    for i in range(1, chromosome_length):
        if i % 2 == 0:
            chromosome.append(random.randint(2, 9))
        else:
            chromosome.append(random.randint(4, 64))
    return chromosome

population_size = 12
generations = 12
number_of_children = 6

population = [generate_random_hyperparameters() for _ in range(population_size)]
scores = []

for individual in population:
    try:
        _, accuracy = create_and_train_model(individual)
        scores.append(accuracy)
    except Exception as e:
        print(f"Error training individual: {e}")
        scores.append(0)

for i in range(generations) :
  create_children(population, scores, number_of_children, i , generations)
  survivor_selection(population, scores, population_size , i , generations)
  print(i + 1  , '/' , generations)
  print('populations : ' , population )
  print('Accuracy : ' , scores)
  print('mean : ' , np.mean(scores))
  print('best : ' , np.max(scores))


1 / 12
populations :  [[2, 5, 6, 19, 5], [3, 51, 3, 47, 3, 41, 8], [2, 5, 3, 17, 5], [6, 42, 2, 39, 5, 49, 8, 18, 9, 41, 6, 55, 2], [3, 48, 8, 25, 6, 13, 5], [5, 9, 6, 57, 7, 40, 5, 49, 3, 6, 5], [4, 9, 5, 59, 3, 28, 6, 33, 7], [3, 27, 7, 17, 6, 48, 3], [2, 18, 2, 55, 7], [4, 44, 8, 38, 6, 13, 9, 34, 2], [2, 5, 6, 57, 5], [2, 9, 3, 17, 7]]
Accuracy :  [0.9872999787330627, 0.9868999719619751, 0.9866999983787537, 0.9898999929428101, 0.9900000095367432, 0.9884999990463257, 0.986299991607666, 0.9889000058174133, 0.9858999848365784, 0.991599977016449, 0.9860000014305115, 0.988099992275238]
mean :  0.9880083252986273
best :  0.991599977016449
2 / 12
populations :  [[2, 5, 6, 19, 5], [3, 51, 3, 47, 3, 41, 8], [2, 5, 3, 17, 5], [6, 42, 2, 39, 5, 49, 8, 18, 9, 41, 6, 55, 2], [3, 48, 8, 25, 6, 13, 5], [5, 9, 6, 57, 7, 40, 5, 49, 3, 6, 5], [4, 9, 5, 59, 3, 28, 6, 33, 7], [4, 44, 8, 38, 6, 13, 9, 34, 2], [6, 42, 6, 19, 5, 49, 8, 18, 9, 41, 6, 55, 2], [2, 5, 2, 39, 5], [2, 5, 3, 47, 5], [3, 42, 6, 